In [1]:
import pandas as pd
import numpy as np
import requests
from arcgis import GIS
import arcpy
from arcgis.features import FeatureLayer
from datetime import datetime
import os

# Connect to TRPA Enterprise GIS Portal
portal_url = "https://maps.trpa.org/portal"
gis = GIS(portal_url)
arcpy.env.workspace = "F:\GIS\PROJECTS\ResearchAnalysis\VHR\Data\VHR_Staging.gdb"
workspace           = r"F:\GIS\PROJECTS\ResearchAnalysis\VHR\Data\VHR_Staging.gdb"
workspace_folder    = r"F:\GIS\PROJECTS\ResearchAnalysis\VHR\VHR"

In [2]:

def get_fs_data(service_url):
    
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features

    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])

    return all_data

def renamecolumns(df, column_mapping, county, jurisdiction):
    df = df.rename(columns=column_mapping).drop(columns=[col for col in df.columns if col not in column_mapping])
    df['county'] = county
    df['jurisdiction'] = jurisdiction
    return df
def download_sdf(service_url):
   
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
    sdf = query_result.sdf
    return sdf



In [3]:
washoe_data_sdf = download_sdf('https://wcgisweb.washoecounty.us/arcgis/rest/services/OpenData/OpenData/MapServer/146')
eldorado_data_sdf = download_sdf('https://see-eldorado.edcgov.us/arcgis/rest/services/Vhr_App/VHR_PERMITS/MapServer/1')
placer_data_sdf = download_sdf('https://services6.arcgis.com/PArfeTGcwA9RGNzN/ArcGIS/rest/services/STR_Permits_231002_Parcels/FeatureServer/0')
CSLT_data_sdf = download_sdf('https://services2.arcgis.com/gWRYLIS16mKUskSO/ArcGIS/rest/services/VHR_Public/FeatureServer/0')
douglas_data_sdf = download_sdf('https://gisservices.douglasnv.us/server/rest/services/VHR_Occupancy/MapServer/67')
CSLT_Hosted_Rental_Data_sdf = download_sdf('https://services2.arcgis.com/gWRYLIS16mKUskSO/arcgis/rest/services/Hosted_Rentals_Public/FeatureServer/0')


In [4]:
washoe_data_sdf.spatial.to_featureclass(os.path.join(workspace, "VHR_Raw_WA"), sanitize_columns=False)
eldorado_data_sdf.spatial.to_featureclass(os.path.join(workspace, "VHR_Raw_EL"), sanitize_columns=False)
placer_data_sdf.spatial.to_featureclass(os.path.join(workspace, "VHR_Raw_PL"), sanitize_columns=False)
CSLT_data_sdf.spatial.to_featureclass(os.path.join(workspace, "VHR_Raw_CSLT"), sanitize_columns=False)
CSLT_Hosted_Rental_Data_sdf.spatial.to_featureclass(os.path.join(workspace, "Hosted_Raw_CSLT"), sanitize_columns=False)
douglas_data_sdf.spatial.to_featureclass(os.path.join(workspace, "VHR_Raw_DG"), sanitize_columns=False)

The operation was attempted on an empty geometry.


'F:\\GIS\\PROJECTS\\ResearchAnalysis\\VHR\\Data\\VHR_Staging.gdb\\VHR_Raw_DG'

In [5]:
washoe_data = get_fs_data('https://wcgisweb.washoecounty.us/arcgis/rest/services/OpenData/OpenData/MapServer/146')
eldorado_data = get_fs_data('https://see-eldorado.edcgov.us/arcgis/rest/services/Vhr_App/VHR_PERMITS/MapServer/1')
placer_data = get_fs_data('https://services6.arcgis.com/PArfeTGcwA9RGNzN/ArcGIS/rest/services/STR_Permits_231002_Parcels/FeatureServer/0')
CSLT__VHR_data = get_fs_data('https://services2.arcgis.com/gWRYLIS16mKUskSO/ArcGIS/rest/services/VHR_Public/FeatureServer/0')
CSLT_Hosted_Rental_Data = get_fs_data('https://services2.arcgis.com/gWRYLIS16mKUskSO/arcgis/rest/services/Hosted_Rentals_Public/FeatureServer/0')
douglas_data = get_fs_data('https://gisservices.douglasnv.us/server/rest/services/VHR_Occupancy/MapServer/67')

In [6]:
#Can't figure out why field mapping is giving an error for this so I had to do a workaround
washoe_data['APN']=washoe_data['B1_PARCEL_NBR']

In [22]:
EL_Field_Mapping = {'APN' : 'APN', 
'License_Status' : 'Status', 
'Account_Number' : 'Permit_ID', 
'Number_of_Overnight_Guests' : 'Max_Occupancy'
}
PL_Field_Mapping = {
   'COPY_STRPermits_231002_ExcelT_4' : 'APN', 
'COPY_STRPermits_231002_ExcelT_2' : 'Status', 
'COPY_STRPermits_231002_ExcelT_1' : 'Permit_ID' 

}
WA_Field_Mapping = {
  'APN' : 'APN', 
'APPL_STATUS' : 'Status', 
'Record_ID' : 'Permit_ID', 
'MaxOccupancy' : 'Max_Occupancy'

}
DG_Field_Mapping = {
  'APN' : 'APN', 
'Permit_Status' : 'Status', 
'Accela_Permit__' : 'Permit_ID', 
'Max_Nighttime_Occupancy' : 'Max_Occupancy',
'Expire_Date': 'expiration_date'
}
CSLT_Field_Mapping = {
    'prcl_id' : 'APN', 

'bl_id' : 'Permit_ID', 
'occpy_max' : 'Max_Occupancy',
'expiration': 'expiration_date'
}
CSLT_Hosted_Mapping = {
    'prcl_id' : 'APN',
    'Permit' : 'Permit_ID',
    'permit_status': 'Status',
    'Max_Occupancy': 'Max_Occupancy'
}


In [23]:
EL_VHR = renamecolumns(eldorado_data, EL_Field_Mapping, 'EL', 'EL')
EL_VHR['APN'] = EL_VHR['APN'].str[:3]+'-'+EL_VHR['APN'].str[3:6]+'-'+EL_VHR['APN'].str[6:9]
PL_VHR = renamecolumns(placer_data, PL_Field_Mapping, 'PL', 'PL')
PL_VHR['APN']=PL_VHR['APN'].str[:11]
DG_VHR = renamecolumns(douglas_data, DG_Field_Mapping, 'DG', 'DG')
DG_VHR['APN']=DG_VHR['APN'].astype(str)
DG_VHR['APN'] = DG_VHR['APN'].str[:4]+'-'+DG_VHR['APN'].str[4:6]+'-'+DG_VHR['APN'].str[6:9]+'-'+DG_VHR['APN'].str[9:12]
DG_VHR['expiration_date']=pd.to_datetime(DG_VHR['expiration_date'], unit='ms')
WA_VHR = renamecolumns(washoe_data, WA_Field_Mapping, 'WA', 'WA')
WA_VHR['APN']=WA_VHR['APN']
CSLT_VHR = renamecolumns(CSLT__VHR_data, CSLT_Field_Mapping, 'EL', 'CSLT')
CSLT_VHR['APN'] = CSLT_VHR['APN']
CSLT_VHR['expiration_date']=pd.to_datetime(CSLT_VHR['expiration_date'], unit='ms')
CSLT_Hosted=renamecolumns(CSLT_Hosted_Rental_Data, CSLT_Hosted_Mapping, 'EL','CSLT')
CSLT_Hosted['Rental_Type']="Hosted Rental"
VHR_Data = pd.concat([EL_VHR, PL_VHR, DG_VHR, WA_VHR, CSLT_VHR, CSLT_Hosted])


In [11]:

def before_or_after_today(date):
    today = datetime.now()
    if date < today:
        return 'expired'
    else:
        return 'active'
VHR_Data['Rental_Type']=VHR_Data['Rental_Type'].fillna('VHR')
VHR_Data.loc[(VHR_Data['jurisdiction']=='CSLT')&(VHR_Data['Status'].isna()), 'Status'] = VHR_Data.loc[(VHR_Data['jurisdiction']=='CSLT')&(VHR_Data['Status'].isna()),'expiration_date'].apply(before_or_after_today)


In [24]:
status_lookup = {'VHR Permit- Waitlist' : 'Pending', 
'VHR Permit- Expired' : 'Inactive', 
'VHR Permit - Active' : 'Active', 
'VHR Permit- Submitted' : 'Pending', 
'VHR Permit- Approved for Payment' : 'Pending', 
'VHR Permit- Incomplete' : 'Pending', 
'VHR Permit - Inactive' : 'Inactive', 
'Inactive' : 'Inactive', 
'VHR Permit- Denied' : 'Inactive', 
'VHR Permit- Withdrawn' : 'Inactive', 
'VHR Permit- Complete' : 'Active', 
'VHR Permit- Closed' : 'Inactive', 
'VHR Permit- Revoked' : 'Inactive', 
'VHR Permit- Suspended' : 'Suspended', 
'current' : 'Active', 
'Current' : 'Active', 
'Suspended' : 'Suspended', 
'Active' : 'Active', 
'About to Expire' : 'Active',
'active':'Active',
'denied': 'Inactive',
'expired': 'Inactive'
}
VHR_Data['Status'] = VHR_Data['Status'].str.strip()
VHR_Data['Status'] = VHR_Data['Status'].replace(status_lookup)
VHR_Data['Status'] = VHR_Data['Status'].fillna('Active')


In [18]:
service_url = 'https://maps.trpa.org/server/rest/services/Parcels/FeatureServer/0'

feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
sdfParcels = query_result.sdf


In [25]:
merged_df = pd.merge(sdfParcels, VHR_Data,  left_on=['APN', 'JURISDICTION'], right_on=['APN', 'jurisdiction'], how='inner' )

In [26]:

drop_columns = ['OBJECTID', 'APO_ADDRESS', 'PSTL_STATE', 'PSTL_ZIP5', 'MAIL_ADD1', 'MAIL_ADD2', 'MAIL_CITY', 'MAIL_STATE', 'MAIL_ZIP5', 'JURISDICTION', 'DESIGN_GUIDELINES_HYPERLINK', 'GlobalID']
merged_df.drop(columns=drop_columns, inplace=True)

In [27]:
## Export spatial dataframes to feature class to use in Spatial join
merged_df.spatial.to_featureclass(os.path.join(workspace, "Parcel_VHR"), sanitize_columns=False)

'F:\\GIS\\PROJECTS\\ResearchAnalysis\\VHR\\Data\\VHR_Staging.gdb\\Parcel_VHR'

In [43]:
merged_df.to_excel('merged.xlsx')
CSLT_VHR.to_excel('cslt.xlsx')
sdfParcels.to_excel('parcels.xlsx')